In [4]:
# default_exp data_processing.time_series

In [1]:
# export
import numpy as np
import pandas as pd
import sys
import os
from scipy import stats

In [6]:
# export
def cast_date(series, to='month', type='str'):
    if to == 'month':
        fmt = '%Y-%m'
        dt_type = 'datetime64[M]'
    elif to == 'day':
        fmt = '%Y-%m-%d'
        dt_type = 'datetime64[D]'
    if type == 'str':
        return pd.to_datetime(series).dt.strftime(fmt)
    elif type == 'datetime':
        return pd.to_datetime(series).astype(dt_type)


def count_time_value(dataframe, ts_col, group_cols, by='day'):
    df = dataframe.copy()
    df[f'ts_{by}'] = cast_date(df[ts_col], to=by)

    groups = [f'ts_{by}']
    groups.extend(group_cols)

    return df.groupby(groups).size().reset_index(name='counts')

In [7]:
#export
def time_over_time(data_frame: pd.DataFrame = None, ts_col: str=None, v_col: str=None
                          , g_col: str=None, shift: int=1, drop_last_value: bool=False)->pd.DataFrame:
    """
    
    return: series that contains time over time
    """
    df = data_frame.copy()
    if g_col:
        last_value_col = 'last_{g_col}_{v_col}'.format(g_col=g_col, v_col=v_col)
        df[last_value_col] = df.groupby(g_col)[v_col].shift(shift)
    else:
        last_value_col = 'last_{v_col}'.format(v_col=v_col)
        df[last_value_col] = df[v_col].shift(shift)

    tot_col = '{value}_{u}o{u}_{shift}'.format(value=v_col, u=ts_col[0], shift=shift)
    df[tot_col] = df[v_col] / df[last_value_col]
    
    return df[tot_col]